In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define the dataset class
class YelpDataset(Dataset):
    def __init__(self, input_ids, token_type_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'token_type_ids': self.token_type_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }


In [2]:
# Load each part of the dataset
data_parts = []
for i in range(1, 6):
    part_data = torch.load(f'yelp_data_part_{i}.pt')
    data_parts.append(part_data)

In [3]:
from torch.utils.data import TensorDataset, DataLoader
# Combine data_parts into a single TensorDataset
full_dataset = TensorDataset(
    torch.cat([part['input_ids'] for part in data_parts]),
    torch.cat([part['token_type_ids'] for part in data_parts]),
    torch.cat([part['attention_mask'] for part in data_parts]),
    torch.cat([part['labels'] for part in data_parts])
)

In [4]:
# Load each part of the dataset
data_parts = []
for i in range(1, 6):
    part_data = torch.load(f'test_yelp_data_part_{i}.pt')
    data_parts.append(part_data)

In [5]:
from torch.utils.data import TensorDataset, DataLoader
# Combine data_parts into a single TensorDataset
test_dataset = TensorDataset(
    torch.cat([part['input_ids'] for part in data_parts]),
    torch.cat([part['token_type_ids'] for part in data_parts]),
    torch.cat([part['attention_mask'] for part in data_parts]),
    torch.cat([part['labels'] for part in data_parts])
)

In [6]:
#subset_indices = range(500)  # Example indices for the subset

#full_dataset = torch.utils.data.Subset(full_dataset, subset_indices)

In [7]:
import torch
from sklearn.model_selection import train_test_split


# Use train_test_split to split the dataset into training, validation, and test sets
train_size = int(0.9 * len(full_dataset))
val_size = (len(full_dataset) - train_size)

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
)

# Define batch size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader
# Fine-tune BERT
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
# Freeze BERT parameters
for param in model.bert.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Specify hyperparameters
epochs = 4
learning_rate = 2e-4
warmup_steps = 3

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
%%capture outs1
# Training loop
for epoch in range(epochs):
    # Training loop with tqdm
    model.train()
    total_train_loss = 0
    total_train_accuracy = 0
    print(f'Epoch {epoch+1}/{epochs}')
    #progress_bar_train = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} (Training)')
    for batch in train_loader:
    #for batch in progress_bar_train:
        input_ids = batch[0].to(device)
        token_type_ids = batch[1].to(device)
        attention_mask = batch[2].to(device)
        labels = batch[3].to(device)
        
        model.zero_grad()
        
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        total_train_loss += loss.item()
        
        # Calculate accuracy
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == labels).float().mean()
        total_train_accuracy += accuracy.item()
        
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        
        #progress_bar_train.set_postfix({'Training Loss': total_train_loss / len(progress_bar_train), 'Training Accuracy': total_train_accuracy / len(progress_bar_train)})

    # Validation loop with tqdm
    model.eval()
    total_val_loss = 0
    total_val_accuracy = 0
    #print(f'Epoch {epoch+1}/{epochs} (Validation)')
    #progress_bar_val = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} (Validation)')
    with torch.no_grad():
        for batch in val_loader:
        #for batch in progress_bar_val:
            input_ids = batch[0].to(device)
            token_type_ids = batch[1].to(device)
            attention_mask = batch[2].to(device)
            labels = batch[3].to(device)

            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            total_val_loss += loss.item()
            
            # Calculate accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            accuracy = (predictions == labels).float().mean()
            total_val_accuracy += accuracy.item()
            
            #progress_bar_val.set_postfix({'Validation Loss': total_val_loss / len(progress_bar_val), 'Validation Accuracy': total_val_accuracy / len(progress_bar_val)})
            
    # Calculate average loss and accuracy
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)
    avg_train_accuracy = total_train_accuracy / len(train_loader)
    
    print(f'Training Loss: {total_train_loss / len(train_loader)}, Training Accuracy: {total_train_accuracy / len(train_loader)}')
    print(f'Validation Loss: {total_val_loss / len(val_loader)}, Validation Accuracy: {total_val_accuracy / len(val_loader)}')

In [ ]:
outs1()

In [ ]:
%%capture outs2
# Testing loop with tqdm
model.eval()
total_test_loss = 0
total_test_accuracy = 0
progress_bar_test = tqdm(test_loader, desc=f'Epoch {epoch+1}/{epochs} (Testing)')
with torch.no_grad():
    for batch in progress_bar_test:
        input_ids = batch[0].to(device)
        token_type_ids = batch[1].to(device)
        attention_mask = batch[2].to(device)
        labels = batch[3].to(device)

        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        total_test_loss += loss.item()
        
        # Calculate accuracy
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == labels).float().mean()
        total_test_accuracy += accuracy.item()
        
        progress_bar_test.set_postfix({'Testing Loss': total_test_loss / len(progress_bar_test),
                                       'Testing Accuracy': total_test_accuracy / len(progress_bar_test)})
    
# Calculate average loss and accuracy
avg_test_loss = total_test_loss / len(test_loader)
avg_test_accuracy = total_test_accuracy / len(test_loader)

# Print the average testing loss and accuracy
print(f'Average Testing Loss: {avg_test_loss:.4f}, Average Testing Accuracy: {avg_test_accuracy:.4f}')

In [ ]:
outs2()

In [ ]:
# Save the model and its parameters
torch.save(model.state_dict(), 'model_3.pth')

In [14]:
outs1()

Epoch 1/4
Training Loss: 1.3043114977590549, Training Accuracy: 0.4369256645881195
Validation Loss: 1.1805259633193335, Validation Accuracy: 0.49444820374015747
Epoch 2/4
Training Loss: 1.2206021236350926, Training Accuracy: 0.4693243354118805
Validation Loss: 1.1485856288061367, Validation Accuracy: 0.504767470472441
Epoch 3/4
Training Loss: 1.207259475223059, Training Accuracy: 0.4743258396236736
Validation Loss: 1.13826560375728, Validation Accuracy: 0.5091043307086615
Epoch 4/4
Training Loss: 1.2029538757560316, Training Accuracy: 0.47701290887211467
Validation Loss: 1.13498492970124, Validation Accuracy: 0.509765625


In [15]:
outs2()

Average Testing Loss: 1.1312, Average Testing Accuracy: 0.5126


Epoch 4/4 (Testing): 100%|##########| 1563/1563 [14:34<00:00,  1.79it/s, Testing Loss=1.13, Testing Accuracy=0.513]   
